# DLE305 
The goal of this notebook is to experiment with convolutional neural networks and update the code to be more readable.



This code cell performs the task of installing the necessary libraries using pip. The > null 2>&1 part redirects both the standard output and standard error to null, effectively silencing any output or error messages.

In [ ]:
!pip install tensorflow > null 2>&1
!pip install seaborn > null 2>&1
!pip install numpy > null 2>&1
!pip install pillow > null 2>&1

This code cell performs the task of importing the necessary libraries.

In [ ]:
import tensorflow as tf
import seaborn as sns
import numpy as np
from PIL import Image
import glob
from collections import defaultdict
from tensorflow.keras import layers, models, optimizers


This code cell performs the task of defining a function `pixels_from_path` that resizes an image to a specified size and converts it to a NumPy array.


In [ ]:
IMG_SIZE = (94, 125)
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    return np_im

This code cell performs the task of using the glob module to get the file paths of all the images in the cats folder.

In [ ]:
glob.glob('cats/*')

This code cell performs the task of counting the occurrences of different image shapes in the cats folder. It uses the pixels_from_path function to get the shape of each image and stores the counts in a defaultdict.

In [ ]:
shape_counts = defaultdict(int)
for i, cat in enumerate(glob.glob('cats/*')[:1000]):
    if i%100==0:
        print(i)
    img_shape = pixels_from_path(cat).shape
    shape_counts[str(img_shape)]= shape_counts[str(img_shape)]+ 1

This code cell performs the task of sorting the shape_counts dictionary items by their counts in descending order and storing the sorted items in the shape_items list.

In [ ]:
shape_items = list(shape_counts.items())
shape_items.sort(key = lambda x: x[1])
shape_items.reverse()

This code cell sets up some parameters for the data processing and model training:

In [ ]:
# 10% of the data will automatically be used for validation
validation_size = 0.1
img_size = IMG_SIZE # resize images to be 374x500 (most common shape)
num_channels = 3 # RGB
sample_size = 8192 #We'll use 8192 pictures (2**13)

This code cell performs the task of loading the training data by counting the number of image files in the cats folder.


In [ ]:
len(glob.glob('cats/*'))

This code cell tests the function pixels_from_path by getting the shape of the image at index 5 in the cats folder.

In [ ]:
pixels_from_path(glob.glob('cats/*')[5]).shape

This code block loads and verifies the shape of an image from the cats folder. It uses the glob module to get file paths, the PIL library to open and resize images, and NumPy to convert images to arrays.

In [ ]:
import glob
from PIL import Image
import numpy as np

IMG_SIZE = (94, 125)

def pixels_from_path(file_path):
    try:
        im = Image.open(file_path)
        im = im.resize(IMG_SIZE)
        np_im = np.array(im)
        return np_im
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

# Check the file paths
file_paths = glob.glob('cats/*')
print(f"Found {len(file_paths)} files.")

# Verify the image file at index 5
if len(file_paths) > 5:
    image_shape = pixels_from_path(file_paths[5])
    if image_shape is not None:
        print(f"Image shape: {image_shape.shape}")
    else:
        print("Failed to process the image.")
else:
    print("Not enough files found.")

This code block loads the training data for cats and dogs by using the pixels_from_path function to process the images and convert them to NumPy arrays. It loads the first 2048 images from the cats and dogs folders.

In [ ]:
SAMPLE_SIZE = 2048
print("loading training cat images...")
cat_train_set = np.asarray([pixels_from_path(cat) for cat in glob.glob('cats/*')[:SAMPLE_SIZE]])
print("loading training dog images...")
dog_train_set = np.asarray([pixels_from_path(dog) for dog in glob.glob('dogs/*')[:SAMPLE_SIZE]])

This code block loads the validation data for cats and dogs by using the pixels_from_path function to process the images and convert them to NumPy arrays. It loads the last 512 images from the cats and dogs folders.

In [ ]:
valid_size = 512
print("loading validation cat images...")
cat_valid_set = np.asarray([pixels_from_path(cat) for cat in glob.glob('cats/*')[-valid_size:]])
print("loading validation dog images...")
dog_valid_set = np.asarray([pixels_from_path(dog) for dog in glob.glob('dogs/*')[-valid_size:]])

This code block is for checking the shape of the training data

In [ ]:
x_train = np.concatenate([cat_train_set, dog_train_set])
labels_train = np.asarray([1 for _ in range(SAMPLE_SIZE)]+[0 for _ in range(SAMPLE_SIZE)])

This code block is for loading the validation data

In [ ]:
x_valid = np.concatenate([cat_valid_set, dog_valid_set])
labels_valid = np.asarray([1 for _ in range(valid_size)]+[0 for _ in range(valid_size)])

This code block is for checking the shape of the training data

In [ ]:
x_train.shape

This code block is for checking the shape of the training data

In [ ]:
labels_train.shape

# Run of the Mill MLP

This code block is for creating a model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

total_pixels = img_size[0] *img_size[1] * 3
fc_size = 512

inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
x = layers.Flatten(name = 'flattened_img')(inputs) #turn image to vector.

x = layers.Dense(fc_size, activation='relu', name='first_layer')(x)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
### Changes:
- lr was not recognised so it was changed to learning_rate

This code block is for compiling the model

In [ ]:
customAdam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["binary_crossentropy"])

### Changes:
- Reshape labels_train to match the model's output shape

this code block is for fitting the model

In [ ]:
# Reshape labels to match the output shape
labels_train = labels_train.reshape(-1, 1)
labels_valid = labels_valid.reshape(-1, 1)

print('Fit model on training data')

history = model.fit(x_train,
                    labels_train,
                    batch_size=32,
                    shuffle=True,
                    epochs=10,
                    validation_data=(x_valid, labels_valid))

# Single Convolutional Layer'
### Changes:
- lr was not recognised so it was changed to learning_rate
this code block is for creating a model

In [ ]:
fc_layer_size = 128
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(24, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)
conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

### Changes:
- lr was not recognised so it was changed to learning_rate

this code block is for compiling the model

In [ ]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

### Changes:
- Reshape labels_train to match the model's output shape

In [ ]:
print('# Fit model on training data')

# Reshape labels_train to match the model's output shape
labels_train = labels_train.reshape(-1, 1)
labels_valid = labels_valid.reshape(-1, 1)

history = conv_model.fit(x_train, 
                    labels_train, # we pass it the labels
                    # If the model is taking forever to train, make this bigger
                    # If it is taking forever to load for the first epoch, make this smaller
                    batch_size=32, 
                    shuffle=True,
                    epochs=5,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(x_valid, labels_valid))

### Changes:
- Flatten the predictions to match the shape of the labels

this code block is for calculating the correlation coefficient

In [ ]:
import numpy as np
import seaborn as sns

# Ensure preds and labels_valid are 1-dimensional
preds = np.asarray(preds).flatten()
labels_valid = np.asarray(labels_valid).flatten()

# Predict values
preds = conv_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])

# Calculate correlation coefficient
np.corrcoef(preds, labels_valid)

This code block is for creating a scatter plot

In [ ]:
sns.scatterplot(x= preds, y= labels_valid)

This code block is for calculating the mean of the predictions

In [ ]:
cat_quantity = sum(labels_valid)

for i in range(1,10):
    print('threshold :'+str(.1*i))
    print(sum(labels_valid[preds > .1*i])/labels_valid[preds > .1*i].shape[0])

this code block is for saving the model

In [ ]:
print(preds.mean())
print(preds[labels_valid == 0].mean())
print(preds[labels_valid == 1].mean())

# Bigger Convolutional Model

This code block is for creating a model

In [ ]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

### Changes:
- lr was not recognised so it was changed to learning_rate

This code block is for compiling the model

In [ ]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

### Changes:
- Reshape labels_train to match the model's output shape

This code block is for fitting the model

In [ ]:
# Fit model on training data
print('# Fit model on training data')

# Reshape labels_train to match the model's output shape
labels_train = labels_train.reshape(-1, 1)
labels_valid = labels_valid.reshape(-1, 1)

history = conv_model.fit(x_train, 
                    labels_train, # we pass it the labels
                    # If the model is taking forever to train, make this bigger
                    # If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle=True,
                    epochs=15,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(x_valid, labels_valid))

This code block is for calculating the correlation coefficient

In [ ]:
import numpy as np
import seaborn as sns

# Ensure preds and labels_valid are 1-dimensional
preds = np.asarray(preds).flatten()
labels_valid = np.asarray(labels_valid).flatten()

# Predict values
preds = conv_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])

# Calculate correlation coefficient
np.corrcoef(preds, labels_valid)

This code block is for creating a scatter plot

In [ ]:
sns.scatterplot(x= preds, y= labels_valid)

This code block is for calculating the mean of the predictions

In [ ]:
print(preds.mean())
print(preds[labels_valid == 0].mean())
print(preds[labels_valid == 1].mean())

This code block calculates and prints the proportion of positive labels (i.e., `labels_valid` equal to 1) for different threshold values applied to the predictions (`preds`). Here's a step-by-step explanation:

1. `cat_quantity = sum(labels_valid)`: This line calculates the total number of positive labels in `labels_valid`.

2. The `for` loop iterates over a range of threshold values from 0.1 to 0.9 (in increments of 0.1).

3. Inside the loop:
   - `print('threshold :'+str(.1*i))`: This line prints the current threshold value.
   - `print(sum(labels_valid[preds > .1*i])/labels_valid[preds > .1*i].shape[0])`: This line calculates and prints the proportion of positive labels for predictions greater than the current threshold. It does this by:
     - Filtering `labels_valid` to include only those entries where the corresponding `preds` value is greater than the current threshold (`.1*i`).
     - Summing the filtered `labels_valid` values to get the count of positive labels.
     - Dividing this count by the total number of filtered entries to get the proportion of positive labels.

The output will show how the proportion of positive labels changes as the threshold increases.

In [ ]:
cat_quantity = sum(labels_valid)

for i in range(1,10):
    print('threshold :'+str(.1*i))
    print(sum(labels_valid[preds > .1*i])/labels_valid[preds > .1*i].shape[0])

In [ ]:
def animal_pic(index):
    return Image.fromarray(x_valid[index])
def cat_index(index):
    return conv_model.predict(np.asarray([x_valid[124]]))[0][0]

### Changes:
- added .keras to the file name

In [ ]:
conv_model.save('conv_model_big.keras')

In [ ]:
index = 600
print("probability of being a cat: {}".format(cat_index(index)))
animal_pic(index)

In [ ]:
conv_model.predict(np.asarray([x_valid[124]]))[0][0]

### Changes:
- added .keras to the file name

In [ ]:
big_model = keras.models.load_model('conv_model_big.keras')

# Huge Model

In [ ]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

huge_conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

### Changes:
- lr was not recognised so it was changed to learning_rate

In [ ]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)
huge_conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

In [ ]:
# Reshape labels_train to match the model's output shape
labels_train = labels_train.reshape(-1, 1)
labels_valid = labels_valid.reshape(-1, 1)

print('# Fit model on training data')

history = huge_conv_model.fit(x_train, 
                    labels_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=5,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(x_valid, labels_valid))

### Changes:
- Flatten the predictions to match the shape of the labels

In [ ]:
import numpy as np

# Ensure preds and labels_valid are 1-dimensional
preds = np.asarray(preds).flatten()
labels_valid = np.asarray(labels_valid).flatten()

preds = huge_conv_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])
np.corrcoef(preds, labels_valid)

In [ ]:
print(preds.mean())
print(preds[labels_valid == 0].mean())
print(preds[labels_valid == 1].mean())

In [ ]:
cat_quantity = sum(labels_valid)

for i in range(1,10):
    print('threshold :'+str(.1*i))
    print(sum(labels_valid[preds > .1*i])/labels_valid[preds > .1*i].shape[0])

### Changes:
- added .keras to the file name

In [ ]:
huge_conv_model.save('conv_model_huge_e13.keras')

In [ ]:
big_model = keras.models.load_model('conv_model_huge_e13.keras')

In [ ]:
preds = big_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])

In [ ]:
sum(labels_valid)

In [ ]:
for i in range(1,10):
    t = .1*i
    print("{:.1f}:".format(t))
    tp = (preds > t)&(labels_valid==1)
    tn = (preds <= t)&(labels_valid==0)
    print(np.sum(np.where(tp|tn, 1, 0))/1024.)

In [ ]:
# Reshape labels_train to match the model's output shape
labels_train = labels_train.reshape(-1, 1)
labels_valid = labels_valid.reshape(-1, 1)

print('# Fit model on training data')

history = big_model.fit(x_train, 
                    labels_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=10,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(x_valid, labels_valid))

In [ ]:
preds = big_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])
for i in range(1,10):
    t = .1*i
    print("{:.1f}:".format(t))
    tp = (preds > t)&(labels_valid==1)
    tn = (preds <= t)&(labels_valid==0)
    print(np.sum(np.where(tp|tn, 1, 0))/1024.)

In [ ]:
big_model.save('conv_model_big_e19.keras')